**CS 5834, Fall 2020: Intro to Urban Computing Project - STUCCO analysis for Gentrification using LA County Data Set**




### Imports and Mount File Directories


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
# modify "customized_path_to_homework", path of folder in drive, where you project is stored
customized_path_to_homework = "/content/drive/My Drive/cs5834/urbcomp_fall2020/project"
file_name = "LosAngeles_typology_output.csv"

sys.path.append(customized_path_to_homework)

In [3]:
%cd $customized_path_to_homework/data

/content/drive/My Drive/cs5834/urbcomp_fall2020/project/data


In [4]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import datetime
import warnings


In [5]:

"""
sys.path.insert(0,'../')
%load_ext autoreload
%autoreload 2
import nyctaxi_helpers
warnings.filterwarnings('ignore')
"""

"\nsys.path.insert(0,'../')\n%load_ext autoreload\n%autoreload 2\nimport nyctaxi_helpers\nwarnings.filterwarnings('ignore')\n"

In [6]:
#pip install --upgrade mlxtend


In [7]:
#ref https://pbpython.com/market-basket-analysis.html


#may need to install mlxtend package - pip install mlxtend in console
import pandas as pd
# #from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import fpgrowth
# from mlxtend.frequent_patterns import association_rules

In [8]:
!wget -nc https://raw.githubusercontent.com/parsahosseini/contrast/master/stucco.py

File ‘stucco.py’ already there; not retrieving.




# **Process Urban Displacement Model Data for Gentrificaiton**


Reference: https://www.urbandisplacement.org/open-source-project

Based on the work https://ww2.arb.ca.gov/sites/default/files/classic//research/apr/past/13-310.pdf?_ga=2.269282425.968981409.1605209194-851335111.1605209194

**Developing a New
Methodology for Analyzing
Potential Displacement**

University of California, Berkeley

Principal Investigator:
Karen Chapple 

https://ww3.arb.ca.gov/research/single-project.php?row_id=65188

## **Obtain the Data**

Data can be found at https://github.com/urban-displacement/displacement-typologies/blob/main/data/outputs/typologies/LosAngeles_typology_output.csv

In [9]:
download_file = ("https://raw.githubusercontent.com/urban-displacement/displacement-typologies/main/data/outputs/typologies/{}".format(file_name))

In [10]:

#!wget -nc $download_file

In [11]:
## Process Data - if data file already processed then can start at Associaton Rules section
# get the Data File

df = pd.read_csv("https://raw.githubusercontent.com/urban-displacement/displacement-typologies/main/data/outputs/typologies/{}".format(file_name), index_col=0)
#df = pd.read_csv("{}/data/{}".format(customized_path_to_homework, file_name), index_col=0)

# sneak peak in the data
df.head(10)


,pop_18,white_18,hh_18,hinc_18,mhval_18,mhval_18_se,mrent_18,mrent_18_se,total_25_18,total_25_col_bd_18,total_25_col_md_18,total_25_col_pd_18,total_25_col_phd_18,tot_units_built_18,units_40_49_built_18,units_39_early_built_18,ohu_18,rhu_18,mmhcosts_18,iinc_18,mov_wc_w_income_18,mov_wc_9000_18,mov_wc_15000_18,mov_wc_25000_18,mov_wc_35000_18,mov_wc_50000_18,mov_wc_65000_18,mov_wc_75000_18,mov_wc_76000_more_18,mov_oc_w_income_18,mov_oc_9000_18,mov_oc_15000_18,mov_oc_25000_18,state_x,county_x,tract_x,mov_oc_35000_18,mov_oc_50000_18,mov_oc_65000_18,mov_oc_75000_18,...,aboverm_pctch_real_mrent_12_18,aboverm_pctch_real_mhval_90_00,aboverm_pctch_real_mrent_90_00,lostli_00,lostli_18,aboverm_pctch_real_hinc_90_00,aboverm_pctch_real_hinc_00_18,aboverm_ch_per_col_90_00,aboverm_ch_per_col_00_18,aboverm_per_units_pre50_18,GEOID,rail,presence_ph_LIHTC,vul_gent_90,vul_gent_00,vul_gent_18,hotmarket_00,hotmarket_18,gent_90_00,gent_90_00_urban,gent_00_18,gent_00_18_urban,dp_PChRent,dp_RentGap,tr_rent_gap,rm_rent_gap,dense,SAE,AdvG,ARE,BE,SMMI,ARG,EOG,OD,OD_loss,LISD,double_counted,typology,typ_cat
0,7222.0,5614.0,2903.0,166383.0,1207600.0,53795.0,2424.0,469.0,5498.0,1712.0,1267.0,500.0,580.0,3102.0,14.0,13.0,1977.0,926.0,2413.0,66532.0,299.0,106.0,0.0,0.0,31.0,27.0,56.0,0.0,79.0,78.0,33.0,0.0,0.0,6.0,73.0,8324.0,0.0,0.0,0.0,17.0,...,1,1,0,0,0,0,1,1,0,0,6073008324,0,False,0,0,0,1.0,1.0,0,0,0,0,1,0,-33.916506,20.207763,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,9.0,['SAE']
1,2053.0,977.0,819.0,77039.0,450600.0,20989.0,1852.0,70.0,1000.0,418.0,201.0,42.0,90.0,869.0,0.0,0.0,164.0,655.0,1869.0,23309.0,440.0,279.0,48.0,53.0,16.0,5.0,4.0,19.0,16.0,89.0,60.0,24.0,5.0,6.0,73.0,8339.0,0.0,0.0,0.0,0.0,...,1,0,1,1,0,0,1,1,1,0,6073008339,0,False,0,0,0,1.0,1.0,0,0,0,0,1,1,261.359486,20.207763,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,7.0,['ARE']
2,7037.0,2161.0,2180.0,112368.0,572300.0,23069.0,2305.0,152.0,5183.0,1305.0,699.0,132.0,191.0,2180.0,0.0,24.0,1758.0,422.0,2230.0,42923.0,156.0,12.0,15.0,20.0,17.0,0.0,27.0,19.0,46.0,153.0,32.0,0.0,53.0,6.0,73.0,8347.0,17.0,24.0,0.0,0.0,...,1,1,1,0,0,1,0,1,0,0,6073008347,0,False,0,0,0,1.0,1.0,0,0,0,0,1,0,-170.758222,20.207763,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,9.0,['SAE']
3,9862.0,2165.0,3200.0,110455.0,596200.0,20543.0,2264.0,62.0,6942.0,2759.0,1094.0,125.0,150.0,3250.0,0.0,0.0,1242.0,1958.0,2283.0,43566.0,1655.0,233.0,60.0,366.0,73.0,270.0,202.0,73.0,378.0,357.0,2.0,102.0,104.0,6.0,73.0,8354.0,0.0,0.0,0.0,0.0,...,0,1,1,0,0,1,0,1,0,0,6073008354,0,False,0,0,0,1.0,0.0,0,0,0,0,1,0,-77.683079,20.207763,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,1.0,9.0,['SAE']
4,6576.0,2761.0,2116.0,63625.0,546400.0,44981.0,1469.0,94.0,4799.0,1080.0,501.0,79.0,91.0,2204.0,42.0,17.0,1328.0,788.0,1531.0,32913.0,468.0,21.0,64.0,115.0,104.0,114.0,0.0,0.0,50.0,59.0,28.0,0.0,15.0,6.0,73.0,8505.0,0.0,16.0,0.0,0.0,...,1,0,0,0,0,0,0,1,1,0,6073008505,0,False,0,0,0,0.0,1.0,0,0,0,0,1,1,368.577539,20.207763,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0,1.0,6.0,['SMMI']
5,7427.0,5283.0,3096.0,71875.0,903800.0,133114.0,1723.0,108.0,5187.0,1682.0,756.0,139.0,206.0,3337.0,21.0,17.0,1450.0,1646.0,1843.0,42448.0,727.0,99.0,35.0,33.0,141.0,231.0,85.0,26.0,77.0,114.0,21.0,0.0,0.0,6.0,73.0,17604.0,41.0,17.0,35.0,0.0,...,0,1,1,0,0,0,0,1,1,0,6073017604,0,False,0,0,0,1.0,1.0,0,0,0,0,1,1,487.450111,20.207763,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0,1.0,6.0,['SMMI']
6,3275.0,1719.0,1589.0,48004.0,542800.0,88825.0,1341.0,99.0,2477.0,517.0,210.0,56.0,39.0,1841.0,25.0,163.0,315.0,1274.0,1306.0,25783.0,345.0,64.0,56.0,53.0,69.0,42.0,0.0,0.0,61.0,96.0,26.0,22.0,20.0,6.0,73.0,18400.0,0.0,9.0,9.0,0.0,...,1,0,0,0,0,0,1,1,1,0,6073018400,1,False,0,1,0,0.0,1.0,0,0,0,1,1,1,202.870590,20.207763,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1,1.0,1.0,['LISD']
7,5619.0,551.0,1820.0,85465.0,431700.0,25394.0,1406.0,368.0,3983.0,644.0,629.0,63.0,65.0,1844.0,0.0,0.0,1560.0,260.0,1820.0,32490.0,207.0,100.0,32.0,0.0,13.0,35.0,27.0,0.0,0.0,21.0,0.0,0.0,21.0,6.0,37.0,543321.0,0.0,0.0,0.0,0.0,...,1,0,1,0,0,

In [12]:
df.columns

Index(['pop_18', 'white_18', 'hh_18', 'hinc_18', 'mhval_18', 'mhval_18_se',
       'mrent_18', 'mrent_18_se', 'total_25_18', 'total_25_col_bd_18',
       ...
       'BE', 'SMMI', 'ARG', 'EOG', 'OD', 'OD_loss', 'LISD', 'double_counted',
       'typology', 'typ_cat'],
      dtype='object', length=305)

In [13]:
"""
# create a working copy of the data to process
df2 = df.copy()

#removed  'inc_cat_medhhinc_encoded18', 'inc_cat_medhhinc_encoded00',
df2 = df2[[ 'inc80_18',
 'inc120_18',
 'inc80_00',
 'inc120_00',
 'inc80_90',
 'low_80120_18',
 'mod_80120_18',
 'high_80120_18',
 'low_pdmt_medhhinc_18',
 'high_pdmt_medhhinc_18',
 'mod_pdmt_medhhinc_18',
 'mix_low_medhhinc_18',
 'mix_mod_medhhinc_18',
 'mix_high_medhhinc_18',

 'low_80120_00',
 'mod_80120_00',
 'high_80120_00',
 'low_pdmt_medhhinc_00',
 'high_pdmt_medhhinc_00',
 'mod_pdmt_medhhinc_00',
 'mix_low_medhhinc_00',
 'mix_mod_medhhinc_00',
 'mix_high_medhhinc_00',

 'per_all_li_90',
 'per_all_li_00',
 'per_all_li_18',
 'per_nonwhite_90',
 'per_nonwhite_00',
 'per_nonwhite_18',
 'per_rent_90',
 'per_rent_00',
 'per_rent_18',
 'per_col_90',
 'per_col_00',
 'per_col_18',
 'per_units_pre50_18',
 'per_limove_18',
 'per_limove_12',
 'lmh_flag_category',
 'pctch_real_mhval_00_18',
 'pctch_real_mrent_12_18',
 'rent_decrease',
 'rent_marginal',
 'rent_increase',
 'rent_rapid_increase',
 'house_decrease',
 'house_marginal',
 'house_increase',
 'house_rapid_increase',
 'tot_decrease',
 'tot_marginal',
 'tot_increase',
 'tot_rapid_increase',
 'change_flag_category',
 'per_ch_zillow_12_18',
 'ab_50pct_ch',
 'ab_90percentile_ch',
 'rent_50pct_ch',
 'rent_90percentile_ch',
 'hv_abrm_ch',
 'rent_abrm_ch',
 'pctch_real_mhval_90_00',
 'pctch_real_mrent_90_00',
 'pctch_real_hinc_90_00',
 'pctch_real_mrent_00_18',
 'pctch_real_hinc_00_18',
 'ch_per_col_90_00',
 'ch_per_col_00_18',
 'ch_per_limove_12_18',
 'aboverm_per_all_li_90',
 'aboverm_per_all_li_00',
 'aboverm_per_all_li_18',
 'aboverm_per_nonwhite_18',
 'aboverm_per_nonwhite_90',
 'aboverm_per_nonwhite_00',
 'aboverm_per_rent_90',
 'aboverm_per_rent_00',
 'aboverm_per_rent_18',
 'aboverm_per_col_90',
 'aboverm_per_col_00',
 'aboverm_per_col_18',
 'aboverm_real_mrent_90',
 'aboverm_real_mrent_00',
 'aboverm_real_mrent_12',
 'aboverm_real_mrent_18',
 'aboverm_real_mhval_90',
 'aboverm_real_mhval_00',
 'aboverm_real_mhval_18',
 'aboverm_pctch_real_mhval_00_18',
 'aboverm_pctch_real_mrent_00_18',
 'aboverm_pctch_real_mrent_12_18',
 'aboverm_pctch_real_mhval_90_00',
 'aboverm_pctch_real_mrent_90_00',
 'lostli_00',
 'lostli_18',
 'aboverm_pctch_real_hinc_90_00',
 'aboverm_pctch_real_hinc_00_18',
 'aboverm_ch_per_col_90_00',
 'aboverm_ch_per_col_00_18',
 'aboverm_per_units_pre50_18',
 'GEOID',
 'rail',
 'presence_ph_LIHTC',
 'hotmarket_00',
 'hotmarket_18',
 'dp_PChRent',
 'dp_RentGap',
 'dense',
 'typology']]


# drop rows which dont have a gentrification category
df2.dropna(subset=['typology'], inplace=True)
df2 = df2.set_index('GEOID')

#fill the nas
df2.fillna(0.0, inplace=True)


df2.head(10)
"""

"\n# create a working copy of the data to process\ndf2 = df.copy()\n\n#removed  'inc_cat_medhhinc_encoded18', 'inc_cat_medhhinc_encoded00',\ndf2 = df2[[ 'inc80_18',\n 'inc120_18',\n 'inc80_00',\n 'inc120_00',\n 'inc80_90',\n 'low_80120_18',\n 'mod_80120_18',\n 'high_80120_18',\n 'low_pdmt_medhhinc_18',\n 'high_pdmt_medhhinc_18',\n 'mod_pdmt_medhhinc_18',\n 'mix_low_medhhinc_18',\n 'mix_mod_medhhinc_18',\n 'mix_high_medhhinc_18',\n\n 'low_80120_00',\n 'mod_80120_00',\n 'high_80120_00',\n 'low_pdmt_medhhinc_00',\n 'high_pdmt_medhhinc_00',\n 'mod_pdmt_medhhinc_00',\n 'mix_low_medhhinc_00',\n 'mix_mod_medhhinc_00',\n 'mix_high_medhhinc_00',\n\n 'per_all_li_90',\n 'per_all_li_00',\n 'per_all_li_18',\n 'per_nonwhite_90',\n 'per_nonwhite_00',\n 'per_nonwhite_18',\n 'per_rent_90',\n 'per_rent_00',\n 'per_rent_18',\n 'per_col_90',\n 'per_col_00',\n 'per_col_18',\n 'per_units_pre50_18',\n 'per_limove_18',\n 'per_limove_12',\n 'lmh_flag_category',\n 'pctch_real_mhval_00_18',\n 'pctch_real_mrent_1

In [14]:
# create a working copy of the data to process
df2 = df.copy()

"""
removed  'inc_cat_medhhinc_encoded18', 'inc_cat_medhhinc_encoded00', 
'inc80_18',
 'inc120_18',
 'inc80_00',
 'inc120_00',
 'inc80_90',
  'per_col_90',
 'per_col_00',
 'per_col_18',
  'tot_decrease',
 'tot_marginal',
 'tot_increase',
 'tot_rapid_increase',
  'pctch_real_mhval_90_00',
 'pctch_real_mrent_90_00',
 'pctch_real_mrent_00_18',
 'per_units_pre50_18',
  
"""

df2.head(10)
df2 = df2[[ 

 'low_80120_18',
 'mod_80120_18',
 'high_80120_18',
 'low_pdmt_medhhinc_18',
 'high_pdmt_medhhinc_18',
 'mod_pdmt_medhhinc_18',
 'mix_low_medhhinc_18',
 'mix_mod_medhhinc_18',
 'mix_high_medhhinc_18',

 'low_80120_00',
 'mod_80120_00',
 'high_80120_00',
 'low_pdmt_medhhinc_00',
 'high_pdmt_medhhinc_00',
 'mod_pdmt_medhhinc_00',
 'mix_low_medhhinc_00',
 'mix_mod_medhhinc_00',
 'mix_high_medhhinc_00',

 'per_all_li_90',
 'per_all_li_00',
 'per_all_li_18',
 'per_nonwhite_90',
 'per_nonwhite_00',
 'per_nonwhite_18',
 'per_rent_90',
 'per_rent_00',
 'per_rent_18',

 
 'per_limove_18',
 'per_limove_12',
 'lmh_flag_category',
 'pctch_real_mhval_00_18',
 'pctch_real_mrent_12_18',
 'rent_decrease',
 'rent_marginal',
 'rent_increase',
 'rent_rapid_increase',
 'house_decrease',
 'house_marginal',
 'house_increase',
 'house_rapid_increase',

 'change_flag_category',
 'per_ch_zillow_12_18',
 'ab_50pct_ch',
 'ab_90percentile_ch',
 'rent_50pct_ch',
 'rent_90percentile_ch',
 'hv_abrm_ch',
 'rent_abrm_ch',

 'pctch_real_hinc_90_00',
 
 'pctch_real_hinc_00_18',
 'ch_per_col_90_00',
 'ch_per_col_00_18',
 'ch_per_limove_12_18',
 'aboverm_per_all_li_90',
 'aboverm_per_all_li_00',
 'aboverm_per_all_li_18',
 'aboverm_per_nonwhite_18',
 'aboverm_per_nonwhite_90',
 'aboverm_per_nonwhite_00',
 'aboverm_per_rent_90',
 'aboverm_per_rent_00',
 'aboverm_per_rent_18',
 'aboverm_per_col_90',
 'aboverm_per_col_00',
 'aboverm_per_col_18',
 'aboverm_real_mrent_90',
 'aboverm_real_mrent_00',
 'aboverm_real_mrent_12',
 'aboverm_real_mrent_18',
 'aboverm_real_mhval_90',
 'aboverm_real_mhval_00',
 'aboverm_real_mhval_18',
 'aboverm_pctch_real_mhval_00_18',
 'aboverm_pctch_real_mrent_00_18',
 'aboverm_pctch_real_mrent_12_18',
 'aboverm_pctch_real_mhval_90_00',
 'aboverm_pctch_real_mrent_90_00',
 'lostli_00',
 'lostli_18',
 'aboverm_pctch_real_hinc_90_00',
 'aboverm_pctch_real_hinc_00_18',
 'aboverm_ch_per_col_90_00',
 'aboverm_ch_per_col_00_18',
 'aboverm_per_units_pre50_18',
 'GEOID',
 'rail',
 'presence_ph_LIHTC',
 'hotmarket_00',
 'hotmarket_18',
 'dp_PChRent',
 'dp_RentGap',
 'dense',
 'typology']]

# drop rows which dont have a gentrification category
df2.dropna(subset=['typology'], inplace=True)
df2 = df2.set_index('GEOID')

#fill the nas
df2.fillna(0.0, inplace=True)


df2.head(10)

,low_80120_18,mod_80120_18,high_80120_18,low_pdmt_medhhinc_18,high_pdmt_medhhinc_18,mod_pdmt_medhhinc_18,mix_low_medhhinc_18,mix_mod_medhhinc_18,mix_high_medhhinc_18,low_80120_00,mod_80120_00,high_80120_00,low_pdmt_medhhinc_00,high_pdmt_medhhinc_00,mod_pdmt_medhhinc_00,mix_low_medhhinc_00,mix_mod_medhhinc_00,mix_high_medhhinc_00,per_all_li_90,per_all_li_00,per_all_li_18,per_nonwhite_90,per_nonwhite_00,per_nonwhite_18,per_rent_90,per_rent_00,per_rent_18,per_limove_18,per_limove_12,lmh_flag_category,pctch_real_mhval_00_18,pctch_real_mrent_12_18,rent_decrease,rent_marginal,rent_increase,rent_rapid_increase,house_decrease,house_marginal,house_increase,house_rapid_increase,...,ch_per_limove_12_18,aboverm_per_all_li_90,aboverm_per_all_li_00,aboverm_per_all_li_18,aboverm_per_nonwhite_18,aboverm_per_nonwhite_90,aboverm_per_nonwhite_00,aboverm_per_rent_90,aboverm_per_rent_00,aboverm_per_rent_18,aboverm_per_col_90,aboverm_per_col_00,aboverm_per_col_18,aboverm_real_mrent_90,aboverm_real_mrent_00,aboverm_real_mrent_12,aboverm_real_mrent_18,aboverm_real_mhval_90,aboverm_real_mhval_00,aboverm_real_mhval_18,aboverm_pctch_real_mhval_00_18,aboverm_pctch_real_mrent_00_18,aboverm_pctch_real_mrent_12_18,aboverm_pctch_real_mhval_90_00,aboverm_pctch_real_mrent_90_00,lostli_00,lostli_18,aboverm_pctch_real_hinc_90_00,aboverm_pctch_real_hinc_00_18,aboverm_ch_per_col_90_00,aboverm_ch_per_col_00_18,aboverm_per_units_pre50_18,rail,presence_ph_LIHTC,hotmarket_00,hotmarket_18,dp_PChRent,dp_RentGap,dense,typology
GEOID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6073008324,0.135421,0.107385,0.757193,0.0,1.0,0.0,0.0,0.0,0.0,0.119943,0.108150,0.771907,0.0,1.0,0.0,0.0,0.0,0.0,0.079270,0.119943,0.135421,0.093494,0.142289,0.222653,0.251780,0.234720,0.318980,0.282219,0.202492,aff_predominantly_HI,0.408463,0.190073,0,0,0,1,0,0,1,0,...,0.079726,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,1,1,0,0,0,False,1.0,1.0,1,0,0,9.0
6073008339,0.420550,0.115488,0.463962,0.0,0.0,0.0,0.0,1.0,0.0,0.408602,0.184386,0.407013,0.0,0.0,0.0,0.0,1.0,0.0,0.491918,0.408602,0.420550,0.212961,0.407345,0.524111,0.681657,0.770335,0.799756,0.827718,0.694186,aff_predominantly_MI,0.507987,0.116779,0,0,1,0,0,0,1,0,...,0.133532,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,1,0,0,1,1,1,0,0,False,1.0,1.0,1,1,0,7.0
6073008347,0.203963,0.145846,0.650191,0.0,1.0,0.0,0.0,0.0,0.0,0.080566,0.135776,0.783658,0.0,1.0,0.0,0.0,0.0,0.0,0.023748,0.080566,0.203963,0.414036,0.573591,0.692909,0.249324,0.218327,0.193578,0.392439,0.501044,aff_mix_high,0.723743,0.178534,0,0,0,1,0,0,0,1,...,-0.108605,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,0,0,1,1,1,0,0,1,0,1,0,0,0,False,1.0,1.0,1,0,0,9.0
6073008354,0.205465,0.158679,0.635856,0.0,1.0,0.0,0.0,0.0,0.0,0.061109,0.126384,0.812507,0.0,1.0,0.0,0.0,0.0,0.0,0.057754,0.061109,0.205465,0.412770,0.546594,0.780470,0.371160,0.362174,0.611875,0.378380,0.189442,aff_predominantly_HI,0.520379,0.032206,0,1,0,0,0,0,1,0,...,0.188938,0,0,0,1,0,0,0,0,1,1,1,1,0,1,1,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,0,0,False,1.0,0.0,1,0,0,9.0
6073008505,0.412422,0.160840,0.426737,0.0,0.0,0.0,0.0,1.0,0.0,0.327182,0.219835,0.452983,0.0,0.0,0.0,0.0,1.0,0.0,0.215439,0.327182,0.412422,0.249399,0.362653,0.580140,0.363551,0.336941,0.372401,0.450786,0.560072,aff_mix_mod,0.804569,0.200929,0,0,0,1,0,0,0,1,...,-0.109286,1,0,1,0,0,0,0,0,0,1,1,1,1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,False,0.0,1.0,1,1,0,6.0
6073017604,0.412958,0.134944,0.452098,0.0,0.0,0.0,0.0,1.0,0.0,0.332965,0.163769,0.503267,0.0,0.0,0.0,0.0,0.0,1.0,0.196232,0.332965,0.412958,0.284718,0.236930,0.288676,0.492885,0.527243,0.531654,0.311711,0.469185,aff_mix_mod,1.036260,0.024589,0,1,0,0,0,0,0,1,...,-0.157474,0,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,1,1,0,0,False,1.0,1.0,1,1,0,6.0
6073018400,0.560880,0.153976,0.285144,1.0,0.0,0.0,0.0,0.0,0.0,0.636059,0.196657,0.167283,1.0,0.0,0.0,0.0,0.0,0.0,0.522115,0.636059,0.560880,0.688229,0.670819,0.475115,0.867849,0.845146,0.801762,0.473432,0.438244,aff_mix_

### Convert select continous variables to categorical and one hot encode 

In [15]:
# # convert already dummy variables from float to int
# dummy_var_list = ['hotmarket_00', 'hotmarket_18', 'low_pdmt_medhhinc_00', 'high_pdmt_medhhinc_00', 'mod_pdmt_medhhinc_00',
#                   'mix_low_medhhinc_00', 'mix_mod_medhhinc_00',  'mix_high_medhhinc_00', 'low_pdmt_medhhinc_18', 'high_pdmt_medhhinc_18',
#                   'mod_pdmt_medhhinc_18', 'mix_low_medhhinc_18', 'mix_mod_medhhinc_18', 'mix_high_medhhinc_18', 'presence_ph_LIHTC',
#                   'typology']
# for dummycol in dummy_var_list:
#   df2.loc[:,(dummycol)] = df2[dummycol].astype(int, copy=False)

In [16]:
# use float variables to represent each categorical variable by dividing into quartiles
def convert_percents_to_dummy_vars(X):

    columns = X.columns

    bin_labels_4 = ['VL', 'L', 'H',  'VH']


    for i in range(0, len(columns)):
        if X[columns[i]].dtype=='float64':
          print("Processing: " + columns[i])

          X[columns[i]] = X[columns[i]].fillna(0)
          cat_label = "{}_cat".format(columns[i])
          dummy_label = "{}_dummy".format(columns[i])
          X[cat_label] = pd.qcut(X[columns[i]], 
                                 4,
                              precision=5,
                              duplicates = 'drop',
                              labels=bin_labels_4)
          X = pd.get_dummies(X, columns=[cat_label], prefix=[dummy_label])
          X = X.drop([columns[i]], axis=1)

    return X

In [17]:
# check if categorical then convert to dummy using label and onehot encoding
def convert_categorical_to_dummy_vars(X):

    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import OneHotEncoder    

    columns = X.columns
    indices = X.index
    new_columns = []
    encoded_x = None

    for i in range(0, len(columns)):
        if X[columns[i]].dtype=='O':
          print("Processing: " + columns[i])


          label_encoder = LabelEncoder()
          le = label_encoder.fit(X[columns[i]].apply(str))
          for class_ in le.classes_:
              new_columns.append("{}_{}".format(columns[i],class_))
              
          feature = le.transform(X[columns[i]].apply(str))
          feature = feature.reshape(X.shape[0], 1)
          onehot_encoder = OneHotEncoder(sparse=False)
          onehot_encoder.fit(feature)
          feature = onehot_encoder.transform(feature)
          if encoded_x is None:
              encoded_x = feature
          else:
              encoded_x = np.concatenate((encoded_x, feature), axis=1)
          X = X.drop(columns[i], axis=1)
    new_columns.extend(X.columns)

    X = pd.DataFrame(np.concatenate((encoded_x,X),axis=1),index=indices,columns=new_columns)
    
    return X


### Convert Percentage (floats) to Quartiles and Dummy Variables 

In [18]:
# # process and save data all converted to one hot encoding
# df2 = convert_percents_to_dummy_vars(df2)





In [19]:
# df2 = convert_categorical_to_dummy_vars(df2)


iii. Save the dataset as a CSV file. The first line of the CSV file should be the attribute
names as described in the previous question. 

In [20]:
# df2.to_csv("{}/data/processed_{}".format(customized_path_to_homework, file_name))

# **STUCCO Rule Calculations**


In [21]:
# load already processed data 
# df3 = pd.read_csv("{}/data/processed_{}".format(customized_path_to_homework, file_name), index_col=0)
# df3 = df3.astype(int)


In [22]:
confidence_cut = 0.8
lift_cut = 1.0
conviction_cut = 0.8
support_cut = 0.65

In [23]:
#ref: https://www.ics.uci.edu/~pazzani/Publications/stucco.pdf
#ref: https://github.com/parsahosseini/contrast/blob/master/README.md

In [24]:
from stucco import ContrastSetLearner
learner = ContrastSetLearner(df2, group_feature='typology')

INFO:root:User-defined feature is 'typology'
INFO:root:Data dimensions: 3,485 x 92
INFO:root:Reading object features (n=3)
INFO:root:Reading numeric features (n=89)
INFO:root:32 features lack value; dropping
INFO:root:New data dimensions: 3,485 x 60
INFO:root:Record count (pre-index): 3,485
INFO:root:Record count (index): 3,483


In [25]:
# derive 3-length combinations of a rule and enumerate their abundance.
learner.learn(max_length=3)

INFO:root:Enumerating 3,483 item-sets, i
INFO:root:i=0; n(matrix)=0
INFO:root:i=200; n(matrix)=370,989
INFO:root:i=400; n(matrix)=410,618
INFO:root:i=600; n(matrix)=424,581
INFO:root:i=800; n(matrix)=428,111
INFO:root:i=1000; n(matrix)=434,629
INFO:root:i=1200; n(matrix)=474,399
INFO:root:i=1400; n(matrix)=497,428
INFO:root:i=1600; n(matrix)=500,660
INFO:root:i=1800; n(matrix)=501,266
INFO:root:i=2000; n(matrix)=501,673
INFO:root:i=2200; n(matrix)=502,997
INFO:root:i=2400; n(matrix)=503,635
INFO:root:i=2600; n(matrix)=504,121
INFO:root:i=2800; n(matrix)=504,366
INFO:root:i=3000; n(matrix)=504,471
INFO:root:i=3200; n(matrix)=504,957
INFO:root:i=3400; n(matrix)=505,167
INFO:root:505,251 contingency matrices made
INFO:root:i=0 matrix not-rules enumerated
INFO:root:i=2500 matrix not-rules enumerated
INFO:root:i=5000 matrix not-rules enumerated
INFO:root:i=7500 matrix not-rules enumerated
INFO:root:i=10000 matrix not-rules enumerated
INFO:root:i=12500 matrix not-rules enumerated
INFO:root:i

505251

In [ ]:
output = learner.score(min_lift=3)

Streaming output truncated to the last 5000 lines.
INFO:root:472013 / 505251: {'rule': ('per_rent_00=>(0.013029315960912051, 0.31162220390253337)', 'lmh_flag_category=>aff_mix_high', 'aboverm_real_mhval_18=>1'), 'group': 'typology=>9.0', 'lift': 8.080554814617232}
INFO:root:472016 / 505251: {'rule': ('per_rent_00=>(0.013029315960912051, 0.31162220390253337)', 'lmh_flag_category=>aff_mix_high', 'aboverm_pctch_real_hinc_00_18=>0'), 'group': 'typology=>9.0', 'lift': 5.995450409463149}
INFO:root:472020 / 505251: {'rule': ('per_rent_00=>(0.013029315960912051, 0.31162220390253337)', 'lmh_flag_category=>aff_mix_high', 'presence_ph_LIHTC=>False'), 'group': 'typology=>9.0', 'lift': 4.316488886240391}
INFO:root:472023 / 505251: {'rule': ('per_rent_00=>(0.013029315960912051, 0.31162220390253337)', 'pctch_real_mhval_00_18=>(0.596919632050351, 0.864841816946165)', 'aboverm_real_mhval_18=>1'), 'group': 'typology=>9.0', 'lift': 4.32829572867887}
INFO:root:472030 / 505251: {'rule': ('per_rent_00=>(0.0

Buffered data was truncated after reaching the output size limit.

In [ ]:
output.to_csv("{}/data/stucco_{}".format(customized_path_to_homework, file_name))

###  Rule Metrics

**Rules with high Lift and Confidence**

ref: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.150.6616&rep=rep1&type=pdf

Confidence ranges from 0 to 1. Confidence is an estimate of Pr(C | A), the
probability of observing C given A. After obtaining a rule set, one can immediatly
use confidence as a basis for classifying one new case x. Of all the rules that
apply to x (i.e., the rules whose antecedent is true in x), we choose the one
with highest confidence. This loosely follows the optimal Bayes classifier

ref: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.150.6616&rep=rep1&type=pdf

Conviction is another measure proposed in [4] to tackle some of the weaknesses of confidence and lift. Unlike lift, conviction is sensitive to rule direction
(conv(A → C) 6= conv(C → A)). Conviction is somewhat inspired in the logical
definition of implication and attempts to measure the degree of implication of
a rule. Conviction is infinite for logical implications (confidence 1), and is 1 if
A and C are independent

ref: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.150.6616&rep=rep1&type=pdf

Lift measures how far from independence are A and C. It ranges within
[0, +∞[. Values close to 1 imply that A and C are independent and the rule
is not interesting. 